# train.ipynb: 用所有的训练集训练决策树
这里需要用到预处理过的数据，以减少代码运算时间。请先跑`waveform.py`。

In [6]:
import multiprocessing
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
from utils import loadData, getNum, getPePerWF, saveData, lossfunc_train, lossfunc_eval
from tqdm import tqdm # 展示进度条
import lightgbm as lgb # 训练决策树使用的包

`waveform.py`预处理后的数据在./train中，文件名格式为`f"final_{i}_wf.h5"`

In [7]:
trainPathRoot = './data/final-'
processedPathRoot = './train/final_'

从预处理的数据中，读取训练两个训练集需要的原始数据，并将18个训练集的数据拼接起来。
1. `intTrainWF`: 波形积分
2. `pointsPerTrainWF`: 超过阈值的点数
3. `pePerTrainWFCalc`: 每个波形对应的PE数计算值
4. `meanPeTimePerTrainWF`: 每个波形对应的PETime的手算平均值
5. `pePerTrainWF`: 每个波形对应的PE数真值
6. `wfIndices`: (n+1,)的ndarray，下标为i表示EventID=i的第一次出现在Waveform表中的下标，下标为n表示Waveform表长度

In [8]:
intTrainWF = np.array([], dtype='<i4') # 读取波形积分
pointsPerTrainWF = np.array([], dtype='<i2') # 读取超过阈值的点数
pePerTrainWFCalc = np.array([], dtype='<i2') # 读取每个波形对应的PE数计算值
meanPeTimePerTrainWF = np.array([], dtype='<f8') # 读取每个波形对应的PETime的手算平均值
pePerTrainWF = np.array([], dtype='<i2') # 读取每个波形对应的PE数真值

wfIndices = np.array([0], dtype=int)
p = np.array([], dtype='<f8')

previousIndex = 0
for i in tqdm(range(2, 20)):
    with h5py.File(f"{processedPathRoot}{i}_wf.h5", 'r') as ipt:
        intTrainWF = np.append(intTrainWF, ipt['Waveform']['intWF'][...])
        pointsPerTrainWF = np.append(pointsPerTrainWF, ipt['Waveform']['pointsPerWF'][...])
        pePerTrainWFCalc = np.append(pePerTrainWFCalc, ipt['Waveform']['pePerWFCalc'][...])
        meanPeTimePerTrainWF = np.append(meanPeTimePerTrainWF, ipt['Waveform']['meanPeTimePerWF'][...])
        pePerTrainWF = np.append(pePerTrainWF, ipt['Waveform']['pePerWF'][...])
    
    trainPET, trainWF, trainPT = loadData(f"./data/final-{i}.h5", 'PT')
    numPEW, wfFakeIndices = getNum(trainWF)
    
    wfIndices = np.append(wfIndices, np.zeros(trainPT.shape[0]))
    p = np.append(p, np.zeros(trainPT.shape[0]))
    
    finalIndex = previousIndex + trainPT.shape[0] + 1
    
    wfIndices[previousIndex:finalIndex] = wfFakeIndices + wfIndices[previousIndex]
    
    p[previousIndex:(finalIndex-1)] = trainPT['p']
    
    previousIndex = finalIndex-1

  0%|          | 0/18 [00:00<?, ?it/s]

Structure of data:
<HDF5 dataset "PETruth": shape (9137011,), type "|V20"> PETruth /PETruth
<HDF5 dataset "ParticleTruth": shape (2000,), type "|V40"> ParticleTruth /ParticleTruth
<HDF5 dataset "Waveform": shape (6041164,), type "|V2008"> Waveform /Waveform


  6%|▌         | 1/18 [01:10<20:06, 70.96s/it]

Structure of data:
<HDF5 dataset "PETruth": shape (9222643,), type "|V20"> PETruth /PETruth
<HDF5 dataset "ParticleTruth": shape (2000,), type "|V40"> ParticleTruth /ParticleTruth
<HDF5 dataset "Waveform": shape (6021458,), type "|V2008"> Waveform /Waveform


 11%|█         | 2/18 [02:35<21:00, 78.76s/it]

...

 89%|████████▉ | 16/18 [18:39<02:15, 67.66s/it]

Structure of data:
<HDF5 dataset "PETruth": shape (9202609,), type "|V20"> PETruth /PETruth
<HDF5 dataset "ParticleTruth": shape (2000,), type "|V40"> ParticleTruth /ParticleTruth
<HDF5 dataset "Waveform": shape (6058586,), type "|V2008"> Waveform /Waveform


 94%|█████████▍| 17/18 [19:49<01:08, 68.18s/it]

Structure of data:
<HDF5 dataset "PETruth": shape (9261569,), type "|V20"> PETruth /PETruth
<HDF5 dataset "ParticleTruth": shape (2000,), type "|V40"> ParticleTruth /ParticleTruth
<HDF5 dataset "Waveform": shape (6085040,), type "|V2008"> Waveform /Waveform


100%|██████████| 18/18 [21:03<00:00, 70.20s/it]


下面的代码和`model.ipynb`是完全一样的，除了训练集、验证集大小不一样。

In [9]:
trainDataForPePerWF = lgb.Dataset(
    np.stack(
        (intTrainWF[:-10000000], pointsPerTrainWF[:-10000000], pePerTrainWFCalc[:-10000000]),
        axis=1
    ),
    label=pePerTrainWF[:-10000000],
)
validationDataForPePerWF = lgb.Dataset(
    np.stack(
        (intTrainWF[-10000000:], pointsPerTrainWF[-10000000:], pePerTrainWFCalc[-10000000:]),
        axis=1
    ),
    label=pePerTrainWF[-10000000:],
    reference=trainDataForPePerWF,
)
trainDataForPePerWF.save_binary('./train/trainPePerWF.bin')
validationDataForPePerWF.save_binary('./train/validPePerWF.bin')

[LightGBM] [Info] Saving data to binary file ./train/trainPePerWF.bin


/home/zhaohaimeng/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')


[LightGBM] [Info] Saving data to binary file ./train/validPePerWF.bin


In [10]:
trainDataForPePerWF = lgb.Dataset('./train/trainPePerWF.bin')
validationDataForPePerWF = lgb.Dataset('./train/validPePerWF.bin', reference=trainDataForPePerWF)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'num_leaves': 2**11,
    'learning_rate': 0.01,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads': 20,
    'max_depth': 20,
}
gbmForPePerWF = lgb.train(
    params,
    trainDataForPePerWF,
    num_boost_round=3000,
    valid_sets=validationDataForPePerWF,
    early_stopping_rounds=100,
)
gbmForPePerWF.save_model('modelPePerWF.txt')

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.465467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's rmse: 1.82572
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 1.80789
[3]	valid_0's rmse: 1.79023
[4]	valid_0's rmse: 1.77276
[5]	valid_0's rmse: 1.75547
[6]	valid_0's rmse: 1.73836
[7]	valid_0's rmse: 1.72142
[8]	valid_0's rmse: 1.70466
[9]	valid_0's rmse: 1.68806
[10]	valid_0's rmse: 1.67164
[11]	valid_0's rmse: 1.65539
[12]	valid_0's rmse: 1.6393
[13]	valid_0's rmse: 1.62338
[14]	valid_0's rmse: 1.60763
[15]	valid_0's rmse: 1.59203
[16]	valid_0's rmse: 1.5766
[17]	valid_0's rmse: 1.56132
[18]	valid_0's rmse: 1.54621
[19]	valid_0's rmse: 1.53124
[20]	valid_0's rmse: 1.51644
[21]	valid_0's rmse: 1.50178
[22]	valid_0's rmse: 1.48728
[23]	valid_0's rmse: 1.47293
[24]	valid_0's rmse: 1.45873
[25]	valid_0's rmse: 1.4

[263]	valid_0's rmse: 0.306831
[264]	valid_0's rmse: 0.306286
[265]	valid_0's rmse: 0.305752
[266]	valid_0's rmse: 0.305227
[267]	valid_0's rmse: 0.304712
[268]	valid_0's rmse: 0.304206
[269]	valid_0's rmse: 0.303709
[270]	valid_0's rmse: 0.303222
[271]	valid_0's rmse: 0.302743
[272]	valid_0's rmse: 0.302273
[273]	valid_0's rmse: 0.301812
[274]	valid_0's rmse: 0.30136
[275]	valid_0's rmse: 0.300916
[276]	valid_0's rmse: 0.30048
[277]	valid_0's rmse: 0.300052
[278]	valid_0's rmse: 0.299632
[279]	valid_0's rmse: 0.29922
[280]	valid_0's rmse: 0.298815
[281]	valid_0's rmse: 0.298418
[282]	valid_0's rmse: 0.298029
[283]	valid_0's rmse: 0.297646
[284]	valid_0's rmse: 0.297271
[285]	valid_0's rmse: 0.296903
[286]	valid_0's rmse: 0.296542
[287]	valid_0's rmse: 0.296188
[288]	valid_0's rmse: 0.29584
[289]	valid_0's rmse: 0.295498
[290]	valid_0's rmse: 0.295164
[291]	valid_0's rmse: 0.294835
[292]	valid_0's rmse: 0.294513
[293]	valid_0's rmse: 0.294197
[294]	valid_0's rmse: 0.293886
[295]	valid_

[529]	valid_0's rmse: 0.278409
[530]	valid_0's rmse: 0.278406
[531]	valid_0's rmse: 0.278404
[532]	valid_0's rmse: 0.278402
[533]	valid_0's rmse: 0.278399
[534]	valid_0's rmse: 0.278397
[535]	valid_0's rmse: 0.278395
[536]	valid_0's rmse: 0.278392
[537]	valid_0's rmse: 0.27839
[538]	valid_0's rmse: 0.278388
[539]	valid_0's rmse: 0.278386
[540]	valid_0's rmse: 0.278384
[541]	valid_0's rmse: 0.278382
[542]	valid_0's rmse: 0.27838
[543]	valid_0's rmse: 0.278378
[544]	valid_0's rmse: 0.278376
[545]	valid_0's rmse: 0.278374
[546]	valid_0's rmse: 0.278373
[547]	valid_0's rmse: 0.278371
[548]	valid_0's rmse: 0.278369
[549]	valid_0's rmse: 0.278368
[550]	valid_0's rmse: 0.278366
[551]	valid_0's rmse: 0.278364
[552]	valid_0's rmse: 0.278363
[553]	valid_0's rmse: 0.278361
[554]	valid_0's rmse: 0.27836
[555]	valid_0's rmse: 0.278358
[556]	valid_0's rmse: 0.278357
[557]	valid_0's rmse: 0.278356
[558]	valid_0's rmse: 0.278354
[559]	valid_0's rmse: 0.278353
[560]	valid_0's rmse: 0.278352
[561]	valid

In [13]:
gbmForPePerWF = lgb.Booster(model_file='./modelPePerWF.txt')
pePerTrainWFFinal = gbmForPePerWF.predict(
    np.stack(
        (intTrainWF, pointsPerTrainWF, pePerTrainWFCalc),
        axis=1
    )
)

In [14]:
splitPePerTrainWFFinal = np.split(pePerTrainWFFinal, wfIndices[1:-1].astype(int))
peTotal = np.empty(p.shape[0])
peMean = np.empty(p.shape[0])
peStd = np.empty(p.shape[0])
for index, pePerTrainWFFinalChunk in enumerate(tqdm(splitPePerTrainWFFinal)):
    peTotal[index] = np.sum(pePerTrainWFFinalChunk)
    peMean[index] = np.mean(pePerTrainWFFinalChunk)
    peStd[index] = np.std(pePerTrainWFFinalChunk)

splitMeanPeTimePerTrainWF = np.split(meanPeTimePerTrainWF, wfIndices[1:-1].astype(int))
peTimeMean = np.empty(p.shape[0])
peTimeStd = np.empty(p.shape[0])
for index, meanPeTimePerTrainWFFinalChunk in enumerate(tqdm(splitMeanPeTimePerTrainWF)):
    peTimeMean[index] = np.nanmean(meanPeTimePerTrainWFFinalChunk)
    peTimeStd[index] = np.nanstd(meanPeTimePerTrainWFFinalChunk)

100%|██████████| 35996/35996 [00:06<00:00, 5542.83it/s]


In [15]:
trainDataForP = lgb.Dataset(
    np.stack(
        (peTotal[:-3600], peMean[:-3600], peStd[:-3600], peTimeMean[:-3600], peTimeStd[:-3600]),
        axis=1
    ),
    label=p[:-3600]
)
validationDataForP = lgb.Dataset(
    np.stack(
        (peTotal[-3600:], peMean[-3600:], peStd[-3600:], peTimeMean[-3600:], peTimeStd[-3600:]),
        axis=1
    ),
    label=p[-3600:],
    reference=trainDataForP
)
trainDataForP.save_binary('./train/trainDataForP.bin')
validationDataForP.save_binary('./train/validDataForP.bin')

[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's rmse: 6.14646	valid_0's custom: 5.37767
Training until validation scores don't improve for 300 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's rmse: 6.08513	valid_0's custom: 5.27082
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's rmse: 6.02441	valid_0's custom: 5.16609
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's rmse: 5.96432	valid_0's custom: 5.06349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's rmse: 5.9048	valid_0's custom: 4.96287
[LightGBM] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[67]	valid_0's rmse: 3.17429	valid_0's custom: 1.43294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[68]	valid_0's rmse: 3.14274	valid_0's custom: 1.40458
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[69]	valid_0's rmse: 3.11152	valid_0's custom: 1.37679
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	valid_0's rmse: 3.0806	valid_0's custom: 1.34954
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	valid_0's rmse: 3.04999	valid_0's custom: 1.32284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	valid_0's rmse: 3.0197	valid_0's custom: 1.29668
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	valid_0's rmse: 2.98972	valid_0's custom: 1.27103
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	valid_0's rm

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[131]	valid_0's rmse: 1.68097	valid_0's custom: 0.401586
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[132]	valid_0's rmse: 1.6645	valid_0's custom: 0.393751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[133]	valid_0's rmse: 1.6482	valid_0's custom: 0.386079
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[134]	valid_0's rmse: 1.63206	valid_0's custom: 0.378556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[135]	valid_0's rmse: 1.61608	valid_0's custom: 0.371179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[136]	valid_0's rmse: 1.60028	valid_0's custom: 0.363959
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[137]	valid_0's rmse: 1.58464	valid_0's custom: 0.35688
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[138

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[194]	valid_0's rmse: 0.914842	valid_0's custom: 0.119214
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	valid_0's rmse: 0.906306	valid_0's custom: 0.11701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[196]	valid_0's rmse: 0.897872	valid_0's custom: 0.114854
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[197]	valid_0's rmse: 0.889514	valid_0's custom: 0.112737
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[198]	valid_0's rmse: 0.881238	valid_0's custom: 0.110661
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[199]	valid_0's rmse: 0.873059	valid_0's custom: 0.108628
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[200]	valid_0's rmse: 0.864965	valid_0's custom: 0.106637
[LightGBM] [Warning] No further splits with positive gain, best gain: 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[258]	valid_0's rmse: 0.520331	valid_0's custom: 0.0392165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[259]	valid_0's rmse: 0.516173	valid_0's custom: 0.0386117
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[260]	valid_0's rmse: 0.512061	valid_0's custom: 0.038017
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[261]	valid_0's rmse: 0.508008	valid_0's custom: 0.0374348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[262]	valid_0's rmse: 0.504003	valid_0's custom: 0.0368658
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[263]	valid_0's rmse: 0.500043	valid_0's custom: 0.0363065
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[264]	valid_0's rmse: 0.496133	valid_0's custom: 0.0357598
[LightGBM] [Warning] No further splits with positive gain, best

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[579]	valid_0's rmse: 0.234262	valid_0's custom: 0.00921494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[580]	valid_0's rmse: 0.234266	valid_0's custom: 0.00921625
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[581]	valid_0's rmse: 0.234251	valid_0's custom: 0.00921558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[582]	valid_0's rmse: 0.234252	valid_0's custom: 0.00921586
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[583]	valid_0's rmse: 0.234241	valid_0's custom: 0.0092163
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[584]	valid_0's rmse: 0.234244	valid_0's custom: 0.00921666
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[585]	valid_0's rmse: 0.234219	valid_0's custom: 0.00921517
[LightGBM] [Warning] No further splits with positive gai

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[640]	valid_0's rmse: 0.234143	valid_0's custom: 0.00924259
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[641]	valid_0's rmse: 0.234143	valid_0's custom: 0.00924203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[642]	valid_0's rmse: 0.234171	valid_0's custom: 0.00924496
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[643]	valid_0's rmse: 0.234168	valid_0's custom: 0.00924465
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[644]	valid_0's rmse: 0.234175	valid_0's custom: 0.00924627
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[645]	valid_0's rmse: 0.234179	valid_0's custom: 0.00924635
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[646]	valid_0's rmse: 0.234208	valid_0's custom: 0.00924889
[LightGBM] [Warning] No further splits with positive ga

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[705]	valid_0's rmse: 0.23477	valid_0's custom: 0.00929622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[706]	valid_0's rmse: 0.234789	valid_0's custom: 0.0092976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[707]	valid_0's rmse: 0.234809	valid_0's custom: 0.00929867
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[708]	valid_0's rmse: 0.23483	valid_0's custom: 0.00930044
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[709]	valid_0's rmse: 0.234848	valid_0's custom: 0.00930135
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[710]	valid_0's rmse: 0.234874	valid_0's custom: 0.00930335
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[711]	valid_0's rmse: 0.234888	valid_0's custom: 0.0093042
[LightGBM] [Warning] No further splits with positive gain, 

[766]	valid_0's rmse: 0.235363	valid_0's custom: 0.00936678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[767]	valid_0's rmse: 0.235366	valid_0's custom: 0.00936746
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[768]	valid_0's rmse: 0.235383	valid_0's custom: 0.00936866
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[769]	valid_0's rmse: 0.235383	valid_0's custom: 0.00936883
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[770]	valid_0's rmse: 0.235385	valid_0's custom: 0.00936912
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[771]	valid_0's rmse: 0.235401	valid_0's custom: 0.00937052
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[772]	valid_0's rmse: 0.235418	valid_0's custom: 0.00937196
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[773]	valid_0's rmse: 0.235413	valid_0's custom: 0.0093

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[832]	valid_0's rmse: 0.235496	valid_0's custom: 0.00942468
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[833]	valid_0's rmse: 0.235484	valid_0's custom: 0.00942511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[834]	valid_0's rmse: 0.235463	valid_0's custom: 0.00942472
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[835]	valid_0's rmse: 0.235455	valid_0's custom: 0.00942525
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[836]	valid_0's rmse: 0.235438	valid_0's custom: 0.00942504
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[837]	valid_0's rmse: 0.235419	valid_0's custom: 0.00942436
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[838]	valid_0's rmse: 0.235408	valid_0's custom: 0.00942488
[LightGBM] [Warning] No further splits with positive ga

In [ ]:
trainDataForP = lgb.Dataset('./train/trainDataForP.bin')
validationDataForP = lgb.Dataset('./train/validDataForP.bin', reference=trainDataForPePerWF)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'num_leaves': 2**11,
    'learning_rate': 0.01,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'verbose': 0,
    'num_threads': 20,
    'max_depth': -1,
}
gbmForP = lgb.train(
    params,
    trainDataForP,
    num_boost_round=3000,
    valid_sets=validationDataForP,
    early_stopping_rounds=300,
    fobj=lossfunc_train,
    feval=lossfunc_eval
)
gbmForP.save_model('modelP.txt')